In [7]:
### PSF_comparison.ipynb
### Author: Emily Ramey
### Compares Blake's PSF simulations from Kappa to Carlos's (both in H-band)

# Useful modules
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
# Image resizing/rescaling/downscaling
from skimage.transform import rescale, resize, downscale_local_mean
# Gaussian fit
from sklearn.mixture import GaussianMixture

In [2]:
### File and folder locations:
root_dir = "/g2/scratch/ao/kapa/psfs/"
blake_folder = root_dir+"maos/ltao_7_7_hy/"
carlos_folder = root_dir+"carlos/gc_sim/"
# Relevant FITS data
blake_center_file = blake_folder+"evlpsfcl_1_x0_y0.fits"
carlos_all_files = carlos_folder+"PSF2D_Hband_NyqSampl_LgsAstRadius7p6arcsec_ZA50_5000iter_4LGSsquare.fits" # my God

In [3]:
### Function(s)
def get_img(data, i=0, j=0):
    """
    Return image block at location i,j in Carlos's full FITS block
    Center coordinates are 0, 0
    i,j range from -5 to +5
    
    """
    # Shift indices to be positive
    i += 5
    j += 5
    
    # Get image at index
    s, L = data.shape # s = side of one img (px), L = total block length (px)
    S = int(np.sqrt(L/s)) # number of images on one side of square
    start_img = s*(S*i+j) # flat array index * px length of one image
    
    return data[:, start_img:start_img+s]

In [6]:
### Read in files and extract center images:
### Assuming all data in first HDU
# Blake
with fits.open(blake_center_file) as f:
    blake_ctr = f[0].data
print("Blake img shape:", blake_ctr.shape) # sanity check 1

# Carlos
with fits.open(carlos_all_files) as f:
    carlos_all = f[0].data

# Extract Carlos's center image because why are they all in the same HDU of the same FITS file
carlos_ctr = get_img(carlos_all)
print("Carlos img shape before:", carlos_ctr.shape) # sanity check 2

### Question: do these two images have the same field of view? If not, how much should I crop?

### Rescale Carlos's image to match Blake's
carlos_ctr = resize(carlos_ctr, blake_ctr.shape)
print("Carlos img shape after:", carlos_ctr.shape) # sanity check 3

Blake img shape: (100, 100)
Carlos img shape before: (120, 120)
Carlos img shape after: (100, 100)


In [10]:
### Get fwhm of each PSF
# Question: is there a Gaussian model we usually use?
# I haven't been able to find a good 2D one just looking online, but I feel like there should be something

# Initialize GM model
gm = GaussianMixture(n_components=1, random_state = np.random.randint(1000)) # using default params
gm.fit(blake_ctr)
gm.covariances_.shape

(1, 100, 100)